# Imports

In [1]:
import json

In [2]:
from collections import OrderedDict

In [3]:
import multiprocessing as mp

In [4]:
import numpy as np
import pandas as pd

from scipy.integrate import solve_ivp

In [5]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

# Model Imports

In [6]:
from models import Model_1
from models import Model_2
from models import Model_3
from models import Model_4

# Auxiliary Functions

In [7]:
from cybernetics.controls.piecewise_constant_controller import PiecewiseConstantController as PWC_Controller
from cybernetics.controls.piecewise_linear_controller import PiecewiseLinearController as PWL_Controller

In [8]:
def create_PWC(time_grid, control_settings):
    return PWC_Controller(time_grid, control_settings, default_value=0.0)

def create_PWL(time_grid, control_settings):
    return PWL_Controller(time_grid, control_settings, default_value=0.0)

In [9]:
def create_task(model_id, controller_type, n_dim, bound, initial_state):
    if model_id == 1:
        model = lambda control: Model_1(control)
    elif model_id == 2:
        model = lambda control: Model_2(control)
    elif model_id == 3:
        model = lambda control: Model_3(control)
    elif model_id == 4:
        model = lambda control: Model_4(control)
    else:
        raise Exception(f"Unsupported model: {model_id}")
    
    temp_model = model(None)
    time_grid = np.linspace(temp_model.t0, temp_model.t1, n_dim + 1)
    accurate_time_grid = np.linspace(temp_model.t0, temp_model.t1, 10 * n_dim + 1)
    search_area = [(-bound, bound) for i in range(n_dim + int(controller_type == "PWL"))]
    
    if controller_type == "PWC":
        controller = lambda control_settings: create_PWC(time_grid, control_settings)
    elif controller_type == "PWL":
        controller = lambda control_settings: create_PWL(time_grid, control_settings)
    else:
        raise Exception(f"Unsupported controller type: {controller_type}")
    
    def f(control_settings):
        control = controller(control_settings)
        m = model(control)
        sol = m.calc_solution(initial_state)
        return m.I(sol) + m.I_term(sol)
    
    def make_dataframe(control_settings):
        control = controller(control_settings)
        m = model(control)
        sol = m.calc_solution(initial_state, accurate_time_grid)
        controls = m.get_controls(sol)
        
        rows = list()
        for t, y, u in list(zip(sol.t, sol.y[:-1, :].T, controls)):
            o = OrderedDict(t=t)
            for i, y_ in enumerate(initial_state):
                o[f"x{i + 1}_initial"] = y_
            for i, y_ in enumerate(y):
                o[f"x{i + 1}"] = y_
            o["u"] = u
            o["t"] = t
            rows.append(o)

        df = pd.DataFrame(rows)
        return df
        
    
    return f, search_area, make_dataframe


# Optimization Tool Parameters

In [10]:
from osol.extremum.algorithms.statistical_anti_gradient_random_search import StatisticalAntiGradientRandomSearch as SAG_RS

In [11]:
sag_rs_radius = 1.0
sag_rs_number_of_samples = 17

# Model Example №1

## Dataframe Generation

In [ ]:
initials_1 = [np.array([v]) for v in np.linspace(-25.0, 25.0, 51)]

In [ ]:
def calc_1(initial_value):
    n_dim = 10
    bound = 100
    working_time = 60 * 30
    
    f, search_area, make_dataframe = create_task(
        model_id=1, 
        controller_type="PWL", 
        n_dim=n_dim, 
        bound=bound,
        initial_state=initial_value)
    
    tool = SAG_RS(radius=sag_rs_radius, number_of_samples=sag_rs_number_of_samples)
    tool.initialize(x=np.array(([0] * len(search_area))), f=f)
    
    suboptimal_control = tool.optimize_max_runtime(
        f, 
        search_area, 
        max_seconds=working_time)
    
    df = make_dataframe(suboptimal_control)
    
    return df, suboptimal_control

In [ ]:
pool = mp.Pool(mp.cpu_count())
dataframes_and_controls_1 = pool.map(calc_1, initials_1)
pool.close()

In [ ]:
dataframe_1 = pd.concat([d for d, _ in dataframes_and_controls_1], axis=0)
dataframe_1.to_csv("results_1.csv", index=False)

with open("controls_1.json", "w") as j:
    controls = [
        OrderedDict(
            initial_state=i_s.tolist(), 
            control=c.tolist()) 
        for i_s, c in zip(initials_1, [c for _, c in dataframes_and_controls_1])]
    json.dump(controls, j)

## Verification of Results

In [29]:
dataframe_1 = pd.read_csv("results_1.csv")
with open("controls_1.json", "r") as j:
    controls = json.load(j)

In [55]:
initial_state = [1.0]

In [56]:
optimal_model = Model_1()
grid = np.linspace(optimal_model.t0, optimal_model.t1, num=101)

In [57]:
optimal_sol = optimal_model.calc_solution(initial_state, grid)
optimal_control = optimal_model.get_controls(optimal_sol)

In [58]:
optimal_I = optimal_model.I(optimal_sol) + optimal_model.I_term(optimal_sol)

In [59]:
controller_settings = [d["control"] for d in controls if d["initial_state"] == initial_state][0]

suboptimal_model = Model_1(
    PWC_Controller(
        time_grid=np.linspace(optimal_model.t0, optimal_model.t1, num=len(controller_settings)),
        values=controller_settings, 
        default_value=0))

In [60]:
suboptimal_sol = suboptimal_model.calc_solution(initial_state, grid)
suboptimal_control = suboptimal_model.get_controls(suboptimal_sol)

In [61]:
suboptimal_I = suboptimal_model.I(suboptimal_sol) + suboptimal_model.I_term(suboptimal_sol)

In [71]:
print(f"Optimal I: {np.round(optimal_I, 5)}")
print(f"Suboptimal I: {np.round(suboptimal_I, 5)}")

Optimal I: 0.25
Suboptimal I: 0.2506


In [67]:
fig_optimal_state = go.Scatter(
    x=optimal_sol.t,
    y=optimal_sol.y[0, :], 
    name="x1 - optimal", 
    yaxis="y1",
    line=dict(color="black"))

query = dataframe_1.query(f"x1_initial == {initial_state[0]}")
fig_suboptimal_state = go.Scatter(
    x=query["t"],
    y=query["x1"], 
    name="x1 - suboptimal", 
    yaxis="y1",
    line=dict(color="black", dash="dash"))

difference = query["x1"] - optimal_sol.y[0, :]
fig_difference_state = go.Scatter(
    x=optimal_sol.t,
    y=difference,
    name="difference", 
    yaxis="y2",
    line=dict(color="black", dash="dot"))

In [68]:
iplot(
    go.Figure(
        data=[
            fig_optimal_state,
            fig_suboptimal_state,
            fig_difference_state
        ],
        layout=go.Layout(
            legend=dict(orientation="h", x=0.1, y=1.1),
            xaxis=dict(
                range=(optimal_model.t0, optimal_model.t1)
            ),
            yaxis1=dict(
                range=(-initial_state[0], initial_state[0])
            ),
            yaxis2=dict(
                range=(-0.2, 0.2),
                overlaying="y1",
                side="right"
            )
        )
    )
)

In [69]:
fig_optimal_control = go.Scatter(
    x=optimal_sol.t,
    y=optimal_control, 
    name="u - optimal", 
    yaxis="y1",
    line=dict(color="black"))

query = dataframe_1.query(f"x1_initial == {initial_state[0]}")
fig_suboptimal_control = go.Scatter(
    x=query["t"],
    y=query["u"], 
    name="u - suboptimal", 
    yaxis="y1",
    line=dict(color="black", dash="dash"))

difference = query["u"] - optimal_control
fig_difference_control = go.Scatter(
    x=optimal_sol.t,
    y=difference,
    name="difference", 
    yaxis="y2",
    line=dict(color="black", dash="dot"))

In [70]:
iplot(
    go.Figure(
        data=[
            fig_optimal_control,
            fig_suboptimal_control,
            fig_difference_control
        ],
        layout=go.Layout(
            legend=dict(orientation="h", x=0.1, y=1.1),
            xaxis=dict(
                range=(optimal_model.t0, optimal_model.t1)
            ),
            yaxis1=dict(
                range=(-initial_state[0], initial_state[0])
            ),
            yaxis2=dict(
                range=(-0.5, 0.5),
                overlaying="y1",
                side="right"
            )
        )
    )
)